### Imports

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

### 2020 Census Population data

In [2]:
url = 'https://www.census.gov/quickfacts/quickfacts/geo/dashboard/US/POP010220'
res = requests.get(url)

In [3]:
soup = BeautifulSoup(res.content)

In [4]:
soup

<!DOCTYPE html>
<html lang="en" prefix="fb: http://www.facebook.com/2008/fbml og: http://opengraphprotocol.org/schema/">
<head>
<!-- build 2563 --><!-- LM 230831 --><meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/>
<meta content="yes" name="mobile-web-app-capable"/>
<meta content="United States,Age and Sex,Race and Hispanic Origin,Population Characteristics,Housing,Families &amp; Living Arrangements,Computer and Internet Use,Education,Health,Economy,Transportation,Income &amp; Poverty,Businesses,Geography" name="keywords"/>
<meta content="U.S. Census Bureau QuickFacts: United States" name="title"/>
<meta content="Frequently requested statistics for: United States" name="description"/>
<meta content="Census Bureau QuickFacts" property="og:site_name"/>
<meta content="U.S. Census Bureau QuickFacts: United Sta

In [5]:
table = soup.find(attrs={'class':'qf-graph-scroll'})
# soup.find_all(attrs={'class':'qf-positive'})
tbody = table.find_all(attrs={'class':"qf-graph-geo"})
tr = tbody[1]
state = tr.find('a').attrs['data-title']
tr.find(attrs={'class':'qf-positive'}).attrs['data-value']

'733391'

In [6]:
pop_2020 = []
for tr in tbody:
    state = tr.find('a').attrs['data-title']
    pop = tr.find(attrs={'class':'qf-positive'}).attrs['data-value']
    
    state_pop = {'State': state, '2020 Population': pop}
    pop_2020.append(state_pop)

In [19]:
pop_2020 = pd.DataFrame(pop_2020)

### 2010 Census Population Data

In [13]:
url2 = 'https://www.census.gov/quickfacts/quickfacts/geo/dashboard/US/POP010210'
res2 = requests.get(url2)

In [14]:
soup2 = BeautifulSoup(res2.content)
table2 = soup2.find(attrs={'class':'qf-graph-scroll'})
tbody2 = table2.find_all(attrs={'class':"qf-graph-geo"})

In [18]:
pop_2010 = []
for tr in tbody2:
    state = tr.find('a').attrs['data-title']
    pop = tr.find(attrs={'class':'qf-positive'}).attrs['data-value']
    
    state_pop = {'State': state, '2010 Population': pop}
    pop_2010.append(state_pop)

pop_2010 = pd.DataFrame(pop_2010)

In [22]:
population = pd.merge(pop_2010, pop_2020)
population

,State,2010 Population,2020 Population
0,Alabama,4779736,5024279
1,Alaska,710231,733391
2,Arizona,6392017,7151502
3,Arkansas,2915918,3011524
4,California,37253956,39538223
5,Colorado,5029196,5773714
6,Connecticut,3574097,3605944
7,Delaware,897934,989948
8,District of Columbia,601723,689545
9,Florida,18801310,21538187


In [23]:
population.to_csv('Data/Population_data_2010_&_2020.csv')